# Milk Bank Logistics Optimization: Inventory Routing Problem (IRP)

## Overview

This notebook implements an Inventory Routing Problem (IRP) optimization model for milk collection logistics.

**Objective:** Minimize total operational costs (Driver Wages + Fuel)

**Method:** Day-by-day planning over a 20-day pilot run (CHUNK_SIZE=1) with two safety triggers:

- If projected inventory would exceed **850 oz**, the depot must be visited that day.
- If a depot has not been picked up in roughly **5 months (~150 days)**, the depot must be visited to meet the shelf-life operating guardrail.

**Tools:** Pyomo (AbstractModel) and GLPK Solver

## 1. Import Libraries and Setup

In [1]:
import pandas as pd
import numpy as np
import os
import ast
from pyomo.environ import *
from pyomo.opt import SolverFactory, TerminationCondition

# Ensure output directory exists
output_dir = '../data/processed'
os.makedirs(output_dir, exist_ok=True)

## 2. Data Loading

Load network data (distances and travel times), forecast data (daily milk volumes), and depot mapping files.

In [2]:
print("Loading processed data...")

# Load network and forecast data
df_network = pd.read_csv('../data/processed/ready_network_data.csv')
df_forecast = pd.read_csv('../data/processed/ready_forecast_data.csv')

# Load depot mapping
id_to_name = {1: "Milk Bank (Hub)"}
mapping_path = '../data/processed/depot_mapping.csv'


df_map = pd.read_csv(mapping_path)
cols = df_map.columns
id_col = next((c for c in cols if 'id' in c.lower()), cols[0])
name_col = next((c for c in cols if 'name' in c.lower() or 'city' in c.lower()), cols[1])
for _, row in df_map.iterrows():
    id_to_name[int(row[id_col])] = str(row[name_col]).strip()
print(f"✅ Mapping loaded from {mapping_path}")

# Transform dataframes to dictionaries
dict_dist = df_network.set_index(['From_ID', 'To_ID'])['Distance_Miles'].to_dict()
dict_time = df_network.set_index(['From_ID', 'To_ID'])['Time_Mins'].to_dict()
dict_forecast = df_forecast.set_index(['Depot_ID', 'Day_Index'])['Volume_oz'].to_dict()

# Define sets
truck_candidate_nodes = sorted(list(df_network['From_ID'].unique()))
depot_nodes = [n for n in truck_candidate_nodes if n != 1]
all_days = sorted(list(df_forecast['Day_Index'].unique()))

print(f"Data Ready. Nodes: {len(truck_candidate_nodes)}")

Loading processed data...
✅ Mapping loaded from ../data/processed/depot_mapping.csv
Data Ready. Nodes: 36


## 3. Mathematical Model Definition

This section defines the Inventory Routing Problem (IRP) model used to create a practical daily pickup plan.

- **Objective:** Minimize operational costs (wages + fuel)
- **Constraints:**
  - Inventory balance
  - Capacity limits (max 1000 oz per depot)
  - Time limits (the daily plan is capped at **660 minutes**)
  - Flow conservation (routes must be consistent)
  - Subtour elimination (prevents disconnected loops)
- **Safety triggers (must-visit rules):**
  - **Overflow protection:** force a visit if projected inventory would exceed **850 oz**
  - **Shelf-life cadence:** force a visit if a depot is due under the ~**5‑month (~150‑day)** operating guardrail


In [3]:
model = AbstractModel()

# Sets
model.N = Set()
model.N_D = Set()
model.T = Set()

# Parameters
model.d = Param(model.N, model.N, default=1000)
model.t_travel = Param(model.N, model.N, default=1000)
model.r = Param(model.N_D, model.T, default=0)

model.C_wage = Param(default=36.0)
model.C_gas = Param(default=0.70)
model.C_depot = Param(default=1000.0)
model.T_max = Param(default=720.0)
model.t_serv = Param(default=15.0)
model.M = Param(default=5000.0)
model.K_buffer = Param(default=150)
model.C_trigger = Param(default=850.0)  # Safety trigger threshold

# Shelf-life (business) guardrail: require pickup if a depot hasn't been serviced in ~5 months
model.H_max_days = Param(default=150.0)
model.must_visit = Param(model.N_D, model.T, default=0)  # 1 if forced visit by shelf-life cadence

# Decision Variables
model.x = Var(model.N, model.N, model.T, domain=Binary)  # Route arcs
model.y = Var(model.N_D, model.T, domain=Binary)  # Visit indicator
model.I = Var(model.N_D, model.T, domain=NonNegativeReals)  # Inventory
model.q = Var(model.N_D, model.T, domain=NonNegativeReals)  # Collection amount
model.u = Var(model.N_D, model.T, domain=NonNegativeReals)  # MTZ subtour elimination

# Objective: Minimize operational costs
def obj_rule(m):
    ops_cost = sum(m.C_wage / 60.0 * (
        sum(m.t_travel[i,j] * m.x[i,j,t] for i in m.N for j in m.N if i!=j) + 
        sum(m.t_serv * m.y[i,t] for i in m.N_D)
    ) for t in m.T) + sum(m.C_gas * m.d[i,j] * m.x[i,j,t] for i in m.N for j in m.N if i!=j for t in m.T)
    return ops_cost
model.Obj = Objective(rule=obj_rule, sense=minimize)

# Constraints
# Inventory balance constraint
def inventory_rule(m, i, t):
    first = min(m.T)
    prev = 0 if t == first else m.I[i, t-1]
    return m.I[i, t] == prev + m.r[i, t] - m.q[i, t]
model.Constraint_Inventory = Constraint(model.N_D, model.T, rule=inventory_rule)

# Force visit rule (if inventory > 850oz, must visit)
def force_visit_rule(m, i, t):
    first = min(m.T)
    prev = 0 if t == first else m.I[i, t-1]
    potential_inv = prev + m.r[i, t]
    return potential_inv - m.C_trigger <= m.M * m.y[i, t]
model.Constraint_ForceVisit = Constraint(model.N_D, model.T, rule=force_visit_rule)

# Shelf-life cadence rule (if must_visit=1, then y must be 1)
def shelf_life_rule(m, i, t):
    return m.y[i, t] >= m.must_visit[i, t]
model.Constraint_ShelfLifeCadence = Constraint(model.N_D, model.T, rule=shelf_life_rule)

# Capacity constraint
def capacity_rule(m, i, t): 
    return m.I[i, t] <= m.C_depot
model.Constraint_Capacity = Constraint(model.N_D, model.T, rule=capacity_rule)

# Time limit constraint
def time_limit_rule(m, t):
    dur = sum(m.t_travel[i,j] * m.x[i,j,t] for i in m.N for j in m.N if i!=j) + \
          sum(m.t_serv * m.y[i,t] for i in m.N_D)
    return dur <= m.T_max
model.Constraint_TimeLimit = Constraint(model.T, rule=time_limit_rule)

# Collection availability constraint
def collection_avail_rule(m, i, t):
    first = min(m.T)
    avail = m.r[i, t] if t == first else m.I[i, t-1] + m.r[i, t]
    return m.q[i, t] <= avail
model.Constraint_Avail = Constraint(model.N_D, model.T, rule=collection_avail_rule)

# Collection link constraint
def collection_link_rule(m, i, t): 
    return m.q[i, t] <= m.M * m.y[i, t]
model.Constraint_Link = Constraint(model.N_D, model.T, rule=collection_link_rule)

# Flow conservation: flow in
def flow_in_rule(m, i, t): 
    return sum(m.x[j, i, t] for j in m.N if j != i) == m.y[i, t]
model.Constraint_FlowIn = Constraint(model.N_D, model.T, rule=flow_in_rule)

# Flow conservation: flow out
def flow_out_rule(m, i, t): 
    return sum(m.x[i, j, t] for j in m.N if j != i) == m.y[i, t]
model.Constraint_FlowOut = Constraint(model.N_D, model.T, rule=flow_out_rule)

# Hub start constraint (at most one route per day)
def hub_start_rule(m, t): 
    return sum(m.x[1, j, t] for j in m.N_D) <= 1
model.Constraint_HubStart = Constraint(model.T, rule=hub_start_rule)

# Hub loop constraint (must return to hub)
def hub_loop_rule(m, t): 
    return sum(m.x[1, j, t] for j in m.N_D) == sum(m.x[j, 1, t] for j in m.N_D)
model.Constraint_HubLoop = Constraint(model.T, rule=hub_loop_rule)

# MTZ subtour elimination constraint
def mtz_rule(m, i, j, t):
    if i == j: 
        return Constraint.Skip
    return m.u[i, t] - m.u[j, t] + len(m.N_D) * m.x[i, j, t] <= len(m.N_D) - 1
model.Constraint_MTZ = Constraint(model.N_D, model.N_D, model.T, rule=mtz_rule)

# Clear-the-Bin Rule (If y=1, I=0; If y=0, I<=1000)
# This constraint effectively handles Capacity AND the requirement to empty the bin.
def clear_inventory_rule(m, i, t):
    return m.I[i, t] <= m.C_depot * (1 - m.y[i, t])
model.Constraint_ClearInventory = Constraint(model.N_D, model.T, rule=clear_inventory_rule)
print("Model formulation complete.")


Model formulation complete.


## 4. Rolling Horizon Optimization

We solve the model **one day at a time** (CHUNK_SIZE=1) over a **20‑day pilot horizon**. After each day, we carry forward the updated depot inventory into the next day’s decision, so the plan reflects what was (or wasn’t) collected.

If the solver cannot find a feasible plan for a day within the time limit, the notebook falls back to a conservative assumption (no trip) and carries inventory forward, rather than stopping the run.

In [4]:
# Configuration
CHUNK_SIZE = 1        # Process one day at a time
TOTAL_DAYS = 20       # Total days to optimize
SOLVER_NAME = 'glpk'  # Solver name

# Initialize results storage
results_routes = []
results_inventory = []

# Initialize inventory (start with 0oz for all depots)
current_inventory = {i: 0.0 for i in depot_nodes}

# Shelf-life cadence state: days since last pickup at each depot
# Business rule: do not let any depot go > ~5 months (150 days) without a pickup.
days_since_pickup = {i: 0 for i in depot_nodes}
MAX_DAYS_WITHOUT_PICKUP = 150

print(f"🚀 Starting Run for {TOTAL_DAYS} days...")

for start_idx in range(0, TOTAL_DAYS, CHUNK_SIZE):
    batch_days = all_days[start_idx : start_idx + CHUNK_SIZE]
    if not batch_days: 
        break
    
    current_day = batch_days[0]
    print(f"Day {current_day}...", end=" ")
    
    # Prepare forecast data (include current inventory)
    batch_forecast = {(i, t): dict_forecast.get((i, t), 0) for i in depot_nodes for t in batch_days}
    for i in depot_nodes:
        batch_forecast[(i, current_day)] += current_inventory[i]

    # Shelf-life cadence trigger (works with day-by-day solving):
    # if a depot has gone MAX_DAYS_WITHOUT_PICKUP-1 days without pickup,
    # force a pickup today.
    must_visit_today = {i: int(days_since_pickup[i] >= (MAX_DAYS_WITHOUT_PICKUP - 1)) for i in depot_nodes}
    batch_must_visit = {(i, t): must_visit_today[i] for i in depot_nodes for t in batch_days}
        
    # Create data dictionary for Pyomo
    data_dict = {
        None: {
            'N': {None: truck_candidate_nodes}, 
            'N_D': {None: depot_nodes}, 
            'T': {None: batch_days},
            'd': dict_dist, 
            't_travel': dict_time, 
            'r': batch_forecast,
            'must_visit': batch_must_visit,
            'H_max_days': {None: float(MAX_DAYS_WITHOUT_PICKUP)},
            'C_wage': {None: 36.0}, 
            'C_gas': {None: 0.70}, 
            'C_depot': {None: 1000.0},
            'T_max': {None: 660.0}, 
            't_serv': {None: 15.0}, 
            'M': {None: 5000.0}, 
            'K_buffer': {None: 150},
            'C_trigger': {None: 850.0}
        }
    }
    
    try:
        # Create model instance and solve
        if 'instance' in locals(): 
            del instance
        instance = model.create_instance(data_dict)
        opt = SolverFactory(SOLVER_NAME)
        opt.options['tmlim'] = 60  # 60 second time limit
        
        results = opt.solve(instance, tee=False, load_solutions=False)

        # Check if solution exists
        has_solution = False
        if (results.solver.status == SolverStatus.ok) or \
           (results.solver.termination_condition == TerminationCondition.optimal) or \
           (results.solver.termination_condition == TerminationCondition.maxTimeLimit):
            try:
                instance.solutions.load_from(results)
                _ = value(instance.Obj)  # Test if solution can be read
                has_solution = True
            except:
                has_solution = False
        
        if has_solution:
            print("✅", end=" ")
            for t in batch_days:
                # Record inventory
                for i in depot_nodes:
                    try:
                        val = value(instance.I[i, t])
                        col = value(instance.q[i, t])
                        current_inventory[i] = val
                        results_inventory.append({'Day': t, 'Depot': i, 'Inv': val, 'Col': col})
                    except:
                        results_inventory.append({'Day': t, 'Depot': i, 'Inv': current_inventory[i], 'Col': 0})

                # Update shelf-life cadence state
                # If a depot is visited today, reset its counter; otherwise increment by 1.
                for i in depot_nodes:
                    try:
                        visited = value(instance.y[i, t]) > 0.5
                    except:
                        visited = False
                    days_since_pickup[i] = 0 if visited else (days_since_pickup[i] + 1)
                
                # Record routes
                edges = []
                for i in truck_candidate_nodes:
                    for j in truck_candidate_nodes:
                        if i == j: 
                            continue
                        try:
                            if value(instance.x[i, j, t]) > 0.5:
                                edges.append((int(i), int(j)))
                        except: 
                            pass
                
                if edges:
                    dist = sum(dict_dist.get((u,v), 0) for u,v in edges)
                    cost = value(instance.Obj)
                    results_routes.append({'Day': t, 'Route': str(edges), 'Miles': dist, 'Cost': cost})
                else:
                    results_routes.append({'Day': t, 'Route': "No Trip", 'Miles': 0, 'Cost': 0})
        else:
            print("❌ (Infeasible)", end=" ")
            # Fallback: Assume no trip, calculate inventory overflow
            results_routes.append({'Day': current_day, 'Route': "Infeasible", 'Miles': 0, 'Cost': 0})
            for i in depot_nodes:
                new_inv = min(current_inventory[i] + dict_forecast.get((i, current_day), 0), 1000.0)
                current_inventory[i] = new_inv
                days_since_pickup[i] = days_since_pickup[i] + 1
                results_inventory.append({'Day': current_day, 'Depot': i, 'Inv': current_inventory[i], 'Col': 0})
            
    except Exception as e:
        print(f"⚠️ Error: {e}", end=" ")
        continue

print("\n🎉 Optimization Complete!")


🚀 Starting Run for 20 days...
Day 1... ✅ Day 2... ✅ Day 3... ✅ Day 4... ✅ Day 5... ✅ Day 6... ✅ Day 7... ✅ Day 8... ✅ Day 9... ✅ Day 10... ✅ Day 11... ✅ Day 12... ✅ Day 13... ✅ Day 14... ✅ Day 15... ✅ Day 16... ✅ Day 17... ✅ Day 18... ✅ Day 19... ✅ Day 20... ✅ 
🎉 Optimization Complete!


## 5. Results Processing and Export

Process route sequences and export results to CSV files.


In [5]:
def order_route_path(edge_input):
    """Convert edge list to ordered route path."""
    if not isinstance(edge_input, str): 
        return [], "No Trip", "No Trip"
    if edge_input in ["No Trip", "Error", "Infeasible", "[]"]: 
        return [], edge_input, edge_input
    try:
        edges = ast.literal_eval(edge_input)
        if not edges: 
            return [], "No Trip", "No Trip"
        next_node = {int(u): int(v) for u, v in edges}
        path = [1]
        curr = 1
        steps = 0
        while steps < len(next_node) + 5:
            if curr in next_node:
                nxt = next_node[curr]
                path.append(nxt)
                curr = nxt
                if curr == 1: 
                    break
            else: 
                break
            steps += 1
        id_str = " -> ".join(str(n) for n in path)
        name_str = " -> ".join([id_to_name.get(n, f"Depot {n}") for n in path])
        return path, id_str, name_str
    except: 
        return [], "Parse Error", "Parse Error"

if results_routes:
    clean_routes = []
    for item in results_routes:
        path, id_seq, name_seq = order_route_path(item['Route'])
        clean_routes.append({
            'Day': item['Day'],
            'Route_Sequence': id_seq,
            'Route_Names': name_seq,
            'Total_Miles': round(float(item['Miles']), 2),
            'Total_Cost': round(float(item['Cost']), 2)
        })
    
    df_routes_clean = pd.DataFrame(clean_routes)
    df_inv_clean = pd.DataFrame(results_inventory)
    
    # Save results
    df_routes_clean.to_csv('../data/processed/final_route_plan.csv', index=False)
    df_inv_clean.to_csv('../data/processed/final_inventory_log.csv', index=False)
    
    print("\n✅ Results Saved!")
    pd.set_option('display.max_colwidth', 150)
    print(df_routes_clean.head(20))
else:
    print("No results.")



✅ Results Saved!
    Day           Route_Sequence  \
0     1                  No Trip   
1     2                  No Trip   
2     3                  No Trip   
3     4                  No Trip   
4     5                  No Trip   
5     6                  No Trip   
6     7                  No Trip   
7     8                  No Trip   
8     9                  No Trip   
9    10              1 -> 6 -> 1   
10   11              1 -> 2 -> 1   
11   12                  No Trip   
12   13    1 -> 8 -> 4 -> 5 -> 1   
13   14              1 -> 3 -> 1   
14   15         1 -> 7 -> 9 -> 1   
15   16                  No Trip   
16   17                  No Trip   
17   18             1 -> 10 -> 1   
18   19  1 -> 15 -> 17 -> 2 -> 1   
19   20                  No Trip   

                                                                                                                                           Route_Names  \
0                                                                      